#SQLCoder-8b
Run the cells below to run inference on our text-to-SQL LLM: SQLCoder-8b.

⭐️ [Github Repo](https://github.com/defog-ai/sqlcoder)

🤗 [Huggingface Page](https://huggingface.co/defog/llama-3-sqlcoder-8b)

##Setup

In [1]:
#!pip install torch transformers bitsandbytes accelerate sqlparse

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
torch.cuda.is_available()

True

In [3]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [4]:
print(available_memory)

12620070912


In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import json

# Modelo de embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

with open("contexto.json", "r", encoding="utf-8") as file:
    ddl_data = json.load(file)

# Criar documentos com as descrições das colunas e tabelas
documents = []
for table_name, table_data in ddl_data["tables"].items():
    table_description = f"Tabela: {table_name} - {table_data['description']}\n"
    columns_description = ""
    
    # Adiciona as descrições das colunas
    for column_name, column_description in table_data["columns"].items():
        columns_description += f"Coluna: {column_name} - {column_description}\n"
    
    # Cria um documento com a tabela e a descrição das colunas
    table_document = table_description + columns_description
    documents.append(Document(page_content=table_document))

# Criar e salvar a base de dados vetorial
db = FAISS.from_documents(documents, embedding_model)
db.save_local("db_faiss")


In [7]:
def retrieve_schema_info(question):
    """Recupera informações relevantes do schema baseado na pergunta."""
    db = FAISS.load_local("db_faiss", embedding_model, allow_dangerous_deserialization=True)
    docs = db.similarity_search(question, k=3)  # Recupera os 3 documentos mais relevantes
    context = "\n".join([doc.page_content for doc in docs])

    return context


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [8]:
model_name = "defog/llama-3-sqlcoder-8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 20e9:
    # if you have atleast 20GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 4 bits – this is slower and less accurate
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_4bit=True,
        device_map="auto",
        use_cache=True,
    )

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [9]:
import sqlparse

def generate_query(question):
    context = retrieve_schema_info(question)
    prompt = """
    <|begin_of_text|><|start_header_id|>user<|end_header_id|>
    
    Please generate an SQL query to answer the following question: `{question}` based on the information below:

    {context}

    Here are some descriptions of the tables and columns:
    - {context}

The model can use these descriptions to understand the columns and their values.
        
    DDL statements:
    CREATE TABLE SIL.DPEDIDOS (
    NUMPEDIDO       VARCHAR2(64) not null,
    NUMPROCESSO     NUMBER(15),
    EPISODIO        NUMBER(20),
    MODULO          VARCHAR2(3),
    DESIGNAMODULO   VARCHAR2(30),
    ESPECIALID      NUMBER(12),
    DESIGNAESPECIAL VARCHAR2(40),
    IDSER           VARCHAR2(10),
    SERVICO         VARCHAR2(60),
    DATAPEDIDO      DATE,
    POSTO           VARCHAR2(20),
    DOCIDDOENTE     NUMBER(12),
    NOMEDOENTE      VARCHAR2(100),
    DATANASCIMENTO  DATE,
    SEXO            VARCHAR2(1),
    MODALIDADE      VARCHAR2(20),
    NUMEXAME        VARCHAR2(64),
    VERIFICADO      NUMBER(1),
    AVISADO         NUMBER(1),
    MARCADO         NUMBER(1), 
    RECOLHIDO       NUMBER(1),
    FECHADO         NUMBER(1),
    SEGURA          NUMBER(1),
    UTILIZA         VARCHAR2(32),
    UTILIZAMARCA    VARCHAR2(32),
    UTILIZARECOLHA  VARCHAR2(32),
    TIPOANALISE     VARCHAR2(1),
    MEDICO          VARCHAR2(40),
    SERV            VARCHAR2(10),
    DATAVERIFICACAO DATE,
    DATAMARCADO     DATE,
    DATAMARCACAO    DATE,
    HORAMARCACAO    DATE,
    DATARECOLHA     DATE,
    DATACHEGADA     DATE,
    DATACHAMADA     DATE,
    DATAEXAME       DATE,
    URGENTE         NUMBER(3),
    OBSERVACOES     VARCHAR2(2000),
    INFCLINICA      VARCHAR2(4000),
    CONTRAINDIC     VARCHAR2(2000),
    SUGMARCACAO     VARCHAR2(2000),
    PEDIDOMAE       VARCHAR2(64),
    POSTOAGENDA     VARCHAR2(30),
    REF_ENTRADA     VARCHAR2(20),
    BLOQAGENDA      NUMBER(1),
    TRIAGEM         VARCHAR2(10)
    PRIMARY KEY (NUMPEDIDO, PEDIDOMAE)
);

    CREATE INDEX SIL.DPEDIDOS_IDX_1 ON SIL.DPEDIDOS (EPISODIO);
    CREATE INDEX SIL.ATM0040 ON SIL.DPEDIDOS (DOCIDDOENTE);
    CREATE INDEX SIL.ATM0044 ON SIL.DPEDIDOS (EPISODIO, IDSER);
    CREATE INDEX SIL.ATM0045 ON SIL.DPEDIDOS (MODULO, IDSER, VERIFICADO, DATAPEDIDO);
    CREATE INDEX SIL.DPUTLI ON SIL.DPEDIDOS (UTILIZAMARCA);
    CREATE INDEX SIL.IDPEDIDOS01 ON SIL.DPEDIDOS (IDSER, NUMPEDIDO, EPISODIO, MODULO, MODALIDADE, MEDICO, MARCADO, DATAPEDIDO);
    CREATE INDEX SIL.IDX_PEDIDOS_POSTOAIDA ON SIL.DPEDIDOS (POSTOAGENDA, IDSER, DATAMARCACAO, FECHADO, MARCADO);
    CREATE INDEX SIL.IDPEDEX ON SIL.DPEDIDOS (NUMEXAME);
    
    CREATE TABLE SIL.PEDIDOS (
        NUMPEDIDO           VARCHAR2(64) NOT NULL,
        ORDEM               NUMBER(4) NOT NULL,
        IDSER               VARCHAR2(10),
        MODALIDADE          VARCHAR2(20),
        CODPROTOCOLO        VARCHAR2(10),
        DESIGNACAOPROTOCOLO VARCHAR2(90),
        CODPROP             VARCHAR2(1),
        VERIFICADO          NUMBER(1),
        DATAVERIFICACAO     DATE,
        PEDIDOFILHO         VARCHAR2(64),
        COD_PEDIDO          VARCHAR2(20),
        DES_PEDIDO          VARCHAR2(200),
        NOTA_LINHAS         VARCHAR2(2000),
        CANCELAR            NUMBER,
        DATAMSG             DATE,
        NUMACTOMED          NUMBER,
        ID_BDNR             VARCHAR2(30),
        ID_REQUISICAO       VARCHAR2(30),
        COD_MCDT            VARCHAR2(51),
        PIN_AGENDAMENTO     VARCHAR2(10),
        PRIMARY KEY (NUMPEDIDO, ORDEM)
    );
    
    CREATE INDEX SIL.ATM0048 ON SIL.PEDIDOS (NUMPEDIDO);
    
    CREATE TABLE SIL.DEXAMES (
        NUMEXAME           VARCHAR2(64) not null,
        VERSAO             NUMBER(4)    not null,
        NUMPROCESSO        NUMBER(15),
        EPISODIO           NUMBER(20),
        MODULO             VARCHAR2(3),
        DESIGNAMODULO      VARCHAR2(30),
        ESPECIALID         NUMBER(7),
        DESIGNAESPECIAL    VARCHAR2(40),
        IDSER              VARCHAR2(10) not null,
        SERVICO            VARCHAR2(100),
        DATAPEDIDO         DATE,
        DATAEXAME          DATE,
        DOCIDDOENTE        NUMBER(12),
        NOMEDOENTE         VARCHAR2(100),
        DATANASCIMENTO     DATE,
        SEXO               VARCHAR2(1),
        PATHRELATORIO      VARCHAR2(100),
        GRUPO              VARCHAR2(20) not null,
        NUMPEDIDO          VARCHAR2(64),
        VERIFICADO         NUMBER(1),
        FACTURADO          NUMBER(1),
        IMAGEM             NUMBER(1),
        RELATORIO          NUMBER(2),
        FECHADO            NUMBER(1),
        SEGURA             NUMBER(1),
        UTILIZA            VARCHAR2(32),
        TIPOANALISE        VARCHAR2(1),
        MEDICO             VARCHAR2(40),
        SERV               VARCHAR2(10),
        DATAVERIFICACAO    DATE,
        DATAFACTURACAO     DATE,
        DATARELAT          DATE,
        DATAFECHO          DATE,
        DATAIMAGEM         DATE,
        SUBSISTEMA         NUMBER(6),
        PROVENIENCIA       NUMBER(2),
        LISTAESTUDOS       VARCHAR2(100),
        CCUSTO             NUMBER(20),
        LISTATECNICOS      VARCHAR2(100),
        POSTO              VARCHAR2(20),
        MODALIDADE         VARCHAR2(20),
        DAUDITADO          DATE,
        DEXAMINADO         DATE,
        NUMEROPROV         VARCHAR2(64),
        DTA_ALTA           DATE,
        NOMEFICH           VARCHAR2(64),
        DATAIMPRESSAO      DATE,
        HORAPEDIDO         VARCHAR2(10),
        HORAEXAME          VARCHAR2(10),
        IDSERORI           VARCHAR2(40),
        NORI               NUMBER(1),
        BODYPART           VARCHAR2(500),
        DTA_CONSULTA       DATE,
        NEXT_ESPECIALIDADE VARCHAR2(50),
        MRELAT             NUMBER,
        DATAVERSAO         DATE,
        PRIVADO            VARCHAR2(1),
        REQ_IDSER          VARCHAR2(100),
        AOUCEXAME          NUMBER(1),
        CONSTRAINT DEXAMES_PK PRIMARY KEY (NUMEXAME, IDSER, GRUPO, VERSAO)
    );
    
    CREATE INDEX SIL.DEXAMES_IDX_1 ON SIL.DEXAMES (PATHRELATORIO);
    CREATE INDEX SIL.DEXAMES_IDX_2 ON SIL.DEXAMES (EPISODIO, VERIFICADO);
    CREATE INDEX SIL.DEXAMES_IDX_3 ON SIL.DEXAMES (IDSER, VERIFICADO);
    CREATE INDEX SIL.DEXAMES_IDX_4 ON SIL.DEXAMES (NUMEXAME);
    CREATE INDEX SIL.DEXAMES_IDX_5 ON SIL.DEXAMES (VERIFICADO);
    CREATE INDEX SIL.DEXAMES_IDX_6 ON SIL.DEXAMES (DOCIDDOENTE);
    CREATE INDEX SIL.DEXAMES_IDX_7 ON SIL.DEXAMES (FACTURADO);
    CREATE INDEX SIL.RELATORIOS ON SIL.DEXAMES (SERVICO, DATAEXAME, RELATORIO);
    
    CREATE TABLE SIL.EXAMES (
        NUMEXAME          VARCHAR2(64) not null,
        VERSAO            NUMBER(4)    not null,
        ORDEM             NUMBER(4)    not null,
        VALIDADE          NUMBER(3),
        IDSER             VARCHAR2(10) not null,
        GRUPO             VARCHAR2(20) not null,
        CODIGIF           VARCHAR2(10),
        DESIGNACAOIGIF    VARCHAR2(200),
        CODIGOANALISE     VARCHAR2(10) not null,
        DESIGNACAOANALISE VARCHAR2(200),
        CODPROP           VARCHAR2(1),
        VERIFICADO        NUMBER(1),
        FACTURADO         NUMBER(1),
        DATAVERIFICACAO   DATE,
        DATAFACTURACAO    DATE,
        TIPOANESTESIA     VARCHAR2(50),
        MEDICOANESTESIA   VARCHAR2(50),
        IDSERORI          VARCHAR2(10),
        ID_BDNR           VARCHAR2(30),
        ID_REQUISICAO     VARCHAR2(30),
        COD_MCDT          VARCHAR2(51),
        PIN_AGENDAMENTO   VARCHAR2(10),
        PRIMARY KEY (IDSER, GRUPO, NUMEXAME, CODIGOANALISE, VERSAO, ORDEM)
    );
    
    CREATE INDEX SIL.CODIGIF ON SIL.EXAMES (CODIGIF);
    
    
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    
    The following SQL query best answers the question `{question}`:
    ```sql
    """
    updated_prompt = prompt.format(question=question, context=context)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=500,
        do_sample=False,
        num_beams=1,
        temperature=0.0,
        top_p=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    # return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)
    return outputs[0].split("```sql")[1].split(";")[0]



##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

In [10]:
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [11]:
question = "quero o nome dos pacientes que sejam do sexo feminino"
generated_sql = generate_query(question)
print(sqlparse.format(generated_sql, reindent=True))

Contexto recuperado: Tabela: SIL.DPEDIDOS - Tabela que armazena pedidos de exames médicos e informações associadas.
Coluna: NUMPEDIDO - Identificador único do pedido.
Coluna: NUMPROCESSO - Número do processo clínico.
Coluna: EPISODIO - Identificador do episódio clínico.
Coluna: MODULO - Código do módulo do episódio.
Coluna: DESIGNAMODULO - Nome do módulo
Coluna: ESPECIALID - Código da especialidade médica associada ao pedido.
Coluna: DESIGNAESPECIAL - Nome da especialidade médica associada ao pedido.
Coluna: IDSER - Código do serviço clínico responsável pela realização do pedido.
Coluna: SERVICO - Nome do serviço clínico responsável pela realização pedido.
Coluna: DATAPEDIDO - Data em que o pedido foi feito.
Coluna: POSTO - Posto clínico onde o pedido foi registado.
Coluna: DOCIDDOENTE - Identificação do paciente associado ao pedido.
Coluna: NOMEDOENTE - Nome completo do paciente.
Coluna: DATANASCIMENTO - Data de nascimento do paciente.
Coluna: SEXO - Sexo do paciente (1 - Masculino, 2

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(



SELECT d.nomedoente
FROM SIL.DPEDIDOS d
WHERE d.sexo = '2'


In [16]:
question = "Quantos ECG foram realizados pelo serviço de cardiologia em 2024"
generated_sql = generate_query(question)
print(sqlparse.format(generated_sql, reindent=True))




Contexto recuperado: Tabela: SIL.EXAMES - Tabela que armazena detalhes sobre exames médicos realizados, incluindo informações sobre verificações e faturação.
Coluna: NUMEXAME - Identificador único do exame.
Coluna: VERSAO - Número da versão do exame, usado para rastreamento de revisões.
Coluna: ORDEM - Número da ordem do exame dentro de um conjunto de exames.
Coluna: VALIDADE - Indica a validade do exame.
Coluna: IDSER - Código do serviço clínico responsável pela realização do exame.
Coluna: GRUPO - Grupo ou categoria do exame.
Coluna: CODIGIF - Código do ato do exame.
Coluna: DESIGNACAOIGIF - Designação do ato conforme a codificação interna.
Coluna: CODIGOANALISE - Código da análise realizada no exame.
Coluna: DESIGNACAOANALISE - Descrição da análise realizada no exame.
Coluna: CODPROP - Código de propriedade do exame.
Coluna: VERIFICADO - Indica se o exame foi verificado (0 - Não, 1 - Sim, 2- Com erro).
Coluna: FACTURADO - Indica se o exame foi faturado (0 - Não, 1 - Sim, 2- Com erro

In [12]:
question = "Quantos pedidos de exame foram agendados para cada médico no último mês"
generated_sql = generate_query(question)
print(sqlparse.format(generated_sql, reindent=True))

Contexto recuperado: Tabela: SIL.EXAMES - Tabela que armazena detalhes sobre exames médicos realizados, incluindo informações sobre verificações e faturação.
Coluna: NUMEXAME - Identificador único do exame.
Coluna: VERSAO - Número da versão do exame, usado para rastreamento de revisões.
Coluna: ORDEM - Número da ordem do exame dentro de um conjunto de exames.
Coluna: VALIDADE - Indica a validade do exame.
Coluna: IDSER - Código do serviço clínico responsável pela realização do exame.
Coluna: GRUPO - Grupo ou categoria do exame.
Coluna: CODIGIF - Código do ato do exame.
Coluna: DESIGNACAOIGIF - Designação do ato conforme a codificação interna.
Coluna: CODIGOANALISE - Código da análise realizada no exame.
Coluna: DESIGNACAOANALISE - Descrição da análise realizada no exame.
Coluna: CODPROP - Código de propriedade do exame.
Coluna: VERIFICADO - Indica se o exame foi verificado (0 - Não, 1 - Sim, 2- Com erro).
Coluna: FACTURADO - Indica se o exame foi faturado (0 - Não, 1 - Sim, 2- Com erro

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



SELECT d.medico,
       COUNT(*) AS num_pedidos
FROM SIL.DPEDIDOS d
JOIN SIL.DEXAMES e ON d.numexame = e.numexame
WHERE d.datapedido >= (CURRENT_DATE - 30)
GROUP BY d.medico


In [13]:
question = "qual é o Identificador único do pedido do paciente joao maria?"
generated_sql = generate_query(question)
print(sqlparse.format(generated_sql, reindent=True))

Contexto recuperado: Tabela: SIL.PEDIDOS - Tabela que armazena detalhes dos pedidos médicos realizados no sistema.
Coluna: NUMPEDIDO - Identificador único do pedido.
Coluna: ORDEM - ordem do código pedido dentro do mesmo NUMPEDIDO.
Coluna: IDSER - Código do serviço clínico responsável pela realização do pedido.
Coluna: MODALIDADE - Modalidade do exame ou procedimento solicitado.
Coluna: CODPROTOCOLO - Código do protocolo associado ao pedido.
Coluna: DESIGNACAOPROTOCOLO - Nome ou descrição do protocolo associado.
Coluna: CODPROP - Código de proposta ou categoria do pedido.
Coluna: VERIFICADO - Indica se o pedido foi verificado (0 - Não, 1 - Sim, 2- Com erro).
Coluna: DATAVERIFICACAO - Data em que o pedido foi verificado.
Coluna: PEDIDOFILHO - Número do pedido filho, caso o pedido seja derivado de outro.
Coluna: COD_PEDIDO - Código do ato associado ao pedido no sistema.
Coluna: DES_PEDIDO - Descrição detalhada do ato.
Coluna: NOTA_LINHAS - Notas ou observações adicionais sobre o pedido.


In [14]:
question = "da me os os números de pedido nao verificados"
generated_sql = generate_query(question)
print(sqlparse.format(generated_sql, reindent=True))

Contexto recuperado: Tabela: SIL.EXAMES - Tabela que armazena detalhes sobre exames médicos realizados, incluindo informações sobre verificações e faturação.
Coluna: NUMEXAME - Identificador único do exame.
Coluna: VERSAO - Número da versão do exame, usado para rastreamento de revisões.
Coluna: ORDEM - Número da ordem do exame dentro de um conjunto de exames.
Coluna: VALIDADE - Indica a validade do exame.
Coluna: IDSER - Código do serviço clínico responsável pela realização do exame.
Coluna: GRUPO - Grupo ou categoria do exame.
Coluna: CODIGIF - Código do ato do exame.
Coluna: DESIGNACAOIGIF - Designação do ato conforme a codificação interna.
Coluna: CODIGOANALISE - Código da análise realizada no exame.
Coluna: DESIGNACAOANALISE - Descrição da análise realizada no exame.
Coluna: CODPROP - Código de propriedade do exame.
Coluna: VERIFICADO - Indica se o exame foi verificado (0 - Não, 1 - Sim, 2- Com erro).
Coluna: FACTURADO - Indica se o exame foi faturado (0 - Não, 1 - Sim, 2- Com erro

In [15]:
questions = [
    "Quantos pedidos de exames foram rececionados no último mês?",
    "Quantos pedidos foram realizados por cada especialidade nos últimos 3 meses?",
    "Qual o número de pedidos urgentes por serviço no último ano?",
    "Quantos pedidos de exames foram cancelados ou anulados no último mês?",
    "Quantos exames foram marcados para um determinado médico nos últimos 15 dias?",
    "Quantos exames foram verificados no último mês?",
    "Quantos exames foram faturados para cada serviço nos últimos 6 meses?",
    "Quantos exames foram realizados para um determinado paciente?",
    "Qual a distribuição de exames por modalidade no último ano?",
    "Quantos exames ainda não foram fechados para cada médico?",
    "Quantos pedidos ainda não foram rececionados e seus exames associados?",
    "Quantos exames estão associados a cada pedido no último mês?",
    "Qual o tempo médio entre o pedido e a realização do exame?",
    "Quantos pedidos tiveram contraindicações clínicas e foram posteriormente cancelados?",
    "Quantos exames de imagem foram realizados para pacientes com mais de 60 anos?"
]

for i, question in enumerate(questions, start=1):  # Enumerar começa de 1
        generated_sql = generate_query(question)  # Gerar a query baseada na questão
        print(f"Questão {i}: {question}")  # Mostrar o número da questão e o texto
        print(sqlparse.format(generated_sql, reindent=True))  
        print("="*80)

Contexto recuperado: Tabela: SIL.EXAMES - Tabela que armazena detalhes sobre exames médicos realizados, incluindo informações sobre verificações e faturação.
Coluna: NUMEXAME - Identificador único do exame.
Coluna: VERSAO - Número da versão do exame, usado para rastreamento de revisões.
Coluna: ORDEM - Número da ordem do exame dentro de um conjunto de exames.
Coluna: VALIDADE - Indica a validade do exame.
Coluna: IDSER - Código do serviço clínico responsável pela realização do exame.
Coluna: GRUPO - Grupo ou categoria do exame.
Coluna: CODIGIF - Código do ato do exame.
Coluna: DESIGNACAOIGIF - Designação do ato conforme a codificação interna.
Coluna: CODIGOANALISE - Código da análise realizada no exame.
Coluna: DESIGNACAOANALISE - Descrição da análise realizada no exame.
Coluna: CODPROP - Código de propriedade do exame.
Coluna: VERIFICADO - Indica se o exame foi verificado (0 - Não, 1 - Sim, 2- Com erro).
Coluna: FACTURADO - Indica se o exame foi faturado (0 - Não, 1 - Sim, 2- Com erro